In [ ]:
# | default_exp codegen

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

from fastcore.test import *
from typing import Optional, Generator
from jinja2 import UndefinedError
from sal.templates import (
    InMemoryTemplateLoader,
    TemplateRenderer,
    InMemoryTemplateLoader,
)
from sal.loaders import xml_to_data

import jupyter_black
import tempfile
import nbdev.showdoc as showdoc

jupyter_black.load()

In [ ]:
# | export
import abc
from typing import Any

from sal.core import Data
from sal.templates import (
    Renderer,
)

# Code Generation

## What code generation means with sal? 

In it's basic form, it will combine xml files converted to `Data` structures, with jinja templates, to render code. Later we will also introduce some frontmatter.

For this, we need a basic structure to work with for generating code. As an example, we'll be working with an hypotetical "model"

In [ ]:
struct: Data = xml_to_data(
    """
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
    """
)

...and the basic templates used with this structure are:

In [ ]:
model = (
    "class {{ name }}Model(models.Model):\n"
    "    {%- for child in children %}\n"
    "    {{ child | render }}\n"
    "    {%- endfor %}\n"
)

field = "{{ name }} = models.{{ type | title }}Field()"

## Code generator I (jinja only)

Now that we can render `jinja2`, we can make a basic code generator

In [ ]:
# | exporti
class SalAction(abc.ABC):
    @property
    @abc.abstractmethod
    def name() -> str:
        pass

    @abc.abstractmethod
    def process_data(self, sal: "Sal", data: Data) -> str:
        pass

    def __str__(self):
        return f"action:{self.name}"


class ToFileAction(SalAction):
    name = "to-file"

    def process_data(self, sal: "Sal", data: Data) -> str:
        rendered = sal.renderer.render(data, template=Renderer.DEFAULT_TEMPLATE)
        to = data.attrs["to"]
        with open(to, "w") as h:
            h.write(rendered)
        return rendered


class ToStringAction(SalAction):
    name = "to-string"

    def process_data(self, sal: "Sal", data: Data) -> str:
        rendered = sal.renderer.render(data, template=Renderer.DEFAULT_TEMPLATE)
        return rendered


class WrapperAction(SalAction):
    name = "wrapper"

    def process_data(self, sal: "Sal", data: Data) -> str:
        return [sal.process(d) for d in data.children]

In [ ]:
# | export
# TODO add support to inject more action into this
class Sal:
    def __init__(self, renderer: Renderer):
        self.renderer = renderer
        self.actions = [ToFileAction(), ToStringAction(), WrapperAction()]

    def pre_process_data(self, data: Data) -> Data:
        for d, _ in data:
            if d.name in self.action_names:
                continue
            # handle front matter

            if hasattr(self.renderer, "get_metadata_for_template"):
                # TODO rename this
                new_attributes = self.renderer.get_metadata_for_template(d.name, d)
                # update attributes
                d.attrs.update(new_attributes)
        return data

    def process_data(self, data: Data) -> str | Any:
        for action in self.actions:
            if data.name == action.name:
                return action.process_data(self, data)
        return self.renderer.process(data)

    def process(self, data: Data) -> str | Any:
        data = self.pre_process_data(data)
        return self.process_data(data)

    @property
    def action_names(self):
        return [action.name for action in self.actions]

It's important to note that a parent should be able the trigger the rendering of his children (this enures the recursive nature of the template rendering). Look at the `model` template for an example:

In [ ]:
model = (
    "class {{ name }}Model(models.Model):\n"
    "    {%- for child in children %}\n"
    "    {{ child | render }}\n"
    "    {%- endfor %}\n"
)

field = "{{ name }} = models.{{ type | title }}Field()"

In [ ]:
# | hide
repository = InMemoryTemplateLoader(
    templates={
        "model": model,
        "field": field,
    }
)
template_renderer = Renderer(repository=repository, renderer=TemplateRenderer())

With this, here's a basic jinja2-based code generator using the hard coded templates:

In [ ]:
sal = Sal(template_renderer)
test_eq(
    sal.process(struct.clone()).strip(),
    """
class UserModel(models.Model):
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
""".strip(),
)

**todo: document to-file**

**todo: document wrapper**

In [ ]:
# | hide
struct2: Data = xml_to_data(
    """
<wrapper>
<wrapper>
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
</wrapper>
</wrapper>"""
)

sal = Sal(template_renderer)
test_eq(
    sal.process(struct2.clone())[0][0].strip(),
    """
class UserModel(models.Model):
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
""".strip(),
)

We are missing one more thing, we need to be able to save the result to a file and we'd like to have that info in the xml and not mess with code to get the job done. So, here's a new struct:

In [ ]:
destination = tempfile.NamedTemporaryFile()

s_file = xml_to_data(
    f"""
<to-file to="{destination.name}">
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
</to-file>"""
)


sal = Sal(template_renderer)
print(sal.process(s_file.clone()))

In [ ]:
# | hide

destination = tempfile.NamedTemporaryFile()

s_file = xml_to_data(
    f"""
<wrapper>
<to-file to="{destination.name}">
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
</to-file>
</wrapper>"""
)

sal = Sal(template_renderer)
result = sal.process(s_file)

with open(destination.name, "r") as h:
    test_eq(
        h.read(),
        """
class UserModel(models.Model):
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
    """.strip(),
    )

> To make this even more powerful, we can use `frontmatter` to embed meta data into the templates themself and merge those with the attributes of the node. 

> To make it even more powerful, the frontmatter can contain any attribute from the struct so it needs to be extracted in a raw formar, rendered and then extracted. But first, we need new templates..

In [ ]:
model = """
---
reference:  "sigla-{{ node.attrs.name | lower }}-model"
---
class {{ name }}Model(models.Model): # {{ reference }}
    {% for child in children -%}
    {{ child | render }}
    {% endfor %}
"""

field = """
---
reference:  "sigla-{{ node.name | lower }}-model"
---
{{ name }} = models.{{ type | title }}Field() 
"""

repository = InMemoryTemplateLoader(
    templates={
        "model": model,
        "field": field,
    }
)
template_renderer2 = Renderer(repository=repository, renderer=TemplateRenderer())

sal = Sal(template_renderer2)
test_eq(
    sal.process(struct.clone()).strip(),
    """
class UserModel(models.Model): # sigla-user-model
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
""".strip(),
)

In [ ]:
# | hide

destination = tempfile.NamedTemporaryFile()
s_file = xml_to_data(
    f"""
    <wrapper>
<to-file to="{destination.name}">
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
</to-file>
    </wrapper>
"""
)

# sal = Sal(template_renderer2)
# sal.pre_process_data(s_file.clone())

In [ ]:
sal = Sal(template_renderer2)
sal.process(s_file)

with open(destination.name, "r") as h:
    test_eq(
        h.read().strip(),
        """
class UserModel(models.Model): # sigla-user-model
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
    """.strip(),
    )

In [ ]:
xml = xml_to_data(
    """
<to-string>
    <W>
        <a/>
        <a/>
        <b/>
    </W>
</to-string>
"""
)


w = """
---
---
{%- for i in node|imports|sum(None, [])|unique %}
{{ i }}
{%- endfor %}


class W:
    {%- for child in children %}
    {{ child | render }}
    {%- endfor %}
    
"""


a = """
---
imports: 
    - from AAA import A
---
a = AAA()
"""
b = """
---
imports: 
    - from BBB import B
---
b = BBB()
"""

repository = InMemoryTemplateLoader(
    templates={
        "W": w,
        "a": a,
        "b": b,
    }
)


def imports(data: Data):
    imports_ = [d.attrs.get("imports") for d, _ in data]
    imports_ = [d for d in imports_ if d]
    return imports_


template_renderer2 = Renderer(
    repository=repository,
    renderer=TemplateRenderer(),
    filters={"imports": imports},
)


sal = Sal(template_renderer2)
res = sal.process(xml)

assert (
    res.strip()
    == """
from AAA import A
from BBB import B


class W:
    a = AAA()
    a = AAA()
    b = BBB()
""".strip()
)

---

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()